In [ ]:
import pandas as pd
import numpy as np

In [ ]:
data = pd.read_csv('/kaggle/input/seizure-dataset/szdata.csv')
df = pd.DataFrame(data)
df

In [ ]:
columns_to_drop = ['utc_timestamp','acc_x','acc_y','acc_z']
df.drop(columns_to_drop, axis=1, inplace=True)

In [ ]:
df

In [ ]:
x = df.iloc[:,0:5]
y = df.iloc[:,-1]

In [ ]:
from sklearn.model_selection import train_test_split
x_train,x_valid,y_train,y_valid = train_test_split(x,y,test_size=0.2,random_state=15)

# FFNN

In [ ]:
from tensorflow import keras
from tensorflow.keras import layers

model = keras.Sequential([
    layers.Dense(4, activation='relu', input_shape=[5]),
    layers.Dense(4, activation='relu'),
    layers.Dense(1, activation='sigmoid'),
])

In [ ]:
model.compile(
    optimizer='adam',
    loss='binary_crossentropy',
    metrics=['binary_accuracy'],
)

In [ ]:
early_stopping = keras.callbacks.EarlyStopping(
    patience=25,
    min_delta=0.001,
    restore_best_weights=True,
)

In [ ]:
history = model.fit(
    x_train, y_train,
    validation_data=(x_valid, y_valid),
    batch_size=52,
    epochs=50,
    callbacks=[early_stopping],
)

In [ ]:
history_df = pd.DataFrame(history.history)

history_df.loc[5:, ['loss', 'val_loss']].plot()
history_df.loc[5:, ['binary_accuracy', 'val_binary_accuracy']].plot()

print(("Best Validation Loss: {:0.4f}" +\
      "\nBest Validation Accuracy: {:0.4f}")\
      .format(history_df['val_loss'].min(),
              history_df['val_binary_accuracy'].max()))

In [ ]:
y_pred1 = model.predict(x_valid)
y_pred_binary = np.round(y_pred1) 

In [ ]:
y_pred1

In [ ]:
from sklearn.metrics import precision_score, recall_score, f1_score

In [ ]:
precision1 = precision_score(y_valid, y_pred_binary)
recall1 = recall_score(y_valid, y_pred_binary)
f11 = f1_score(y_valid, y_pred_binary)

print(f'Precision: {precision1:.4f}')
print(f'Recall: {recall1:.4f}')
print(f'F1 Score: {f11:.4f}')

# MLP

In [ ]:
from sklearn.neural_network import MLPClassifier

mlp = MLPClassifier(hidden_layer_sizes=(9,8),
                    random_state=15,
                    verbose=True,
                    learning_rate_init=0.01)

mlp.fit(x_train,y_train)

In [ ]:
ypred2=mlp.predict(x_valid)


from sklearn.metrics import classification_report

classification_report(y_valid,ypred2)

In [ ]:
print(accuracy_score(y_valid,ypred2))
print(precision_score(y_valid,ypred2))
print(recall_score(y_valid,ypred2))
print(f1_score(y_valid,ypred2))

# RNN

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import SimpleRNN, Dense, Dropout
seq_length = 5
model_rnn = Sequential()
model_rnn.add(SimpleRNN(64, input_shape=(seq_length, 1), return_sequences=True))
model_rnn.add(Dropout(0.2)) 
model_rnn.add(SimpleRNN(32, return_sequences=True))
model_rnn.add(Dropout(0.2))
model_rnn.add(SimpleRNN(32))  
model_rnn.add(Dropout(0.2))
model_rnn.add(Dense(1, activation='sigmoid')) 

model_rnn.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model_rnn.summary()

In [ ]:
history_1 = model_rnn.fit(
    x_train, y_train,
    validation_data=(x_valid, y_valid),
    batch_size=52,
    epochs=50,
    callbacks=[early_stopping]
)

In [ ]:
history_df_1 = pd.DataFrame(history_1.history)
# Start the plot at epoch 5
history_df_1.loc[5:, ['loss', 'val_loss']].plot()
history_df_1.loc[5:, ['accuracy', 'val_accuracy']].plot()

print(("Best Validation Loss: {:0.4f}" +\
      "\nBest Validation Accuracy: {:0.4f}")\
      .format(history_df_1['val_loss'].min(),
              history_df_1['val_accuracy'].max()))

In [ ]:
y_pred2 = model_rnn.predict(x_valid)
y_pred_binary2 = np.round(y_pred2) 

In [ ]:
precision2 = precision_score(y_valid, y_pred_binary2)
recall2 = recall_score(y_valid, y_pred_binary2)
f12 = f1_score(y_valid, y_pred_binary2)

print(f'Precision: {precision2:.4f}')
print(f'Recall: {recall2:.4f}')
print(f'F1 Score: {f12:.4f}')

In [ ]:
acc2 = accuracy_score(y_valid, y_pred_binary2)
print(f'Accuracy: {acc2:.4f}')

# XGBoost

In [ ]:
import xgboost as xgb

In [ ]:
hyperparameters = {
    'n_estimators': 200,
    'max_depth': 5,
    'learning_rate': 0.1,
}

In [ ]:
modelx = xgb.XGBClassifier(**hyperparameters)
modelx.fit(x_train, y_train)

In [ ]:
y_predx = model.predict(x_valid)

In [ ]:
y_predx

In [ ]:
threshold = 0.5  

y_pred_binaryx = np.where(y_predx >= threshold, 1, 0)

In [ ]:
from sklearn.metrics import accuracy_score

In [ ]:
accuracyx = accuracy_score(y_valid, y_pred_binaryx)
precisionx = precision_score(y_valid, y_pred_binaryx)
recallx = recall_score(y_valid, y_pred_binaryx)
f1x = f1_score(y_valid, y_pred_binaryx)

print(f'Accuracy: {accuracyx:.4f}')
print(f'Precision: {precisionx:.4f}')
print(f'Recall: {recallx:.4f}')
print(f'F1 Score: {f1x:.4f}')

# KNN

In [ ]:
from sklearn.neighbors import KNeighborsClassifier  
knn= KNeighborsClassifier(n_neighbors=5000)  
knn.fit(x_train, y_train) 

In [ ]:
knn_pred= knn.predict(x_valid)  

In [ ]:
print(classification_report(y_valid,knn_pred))
accuracy = accuracy_score(y_valid, knn_pred)
print("Accuracy = ",accuracy)

In [ ]:
print(precision_score(y_valid, knn_pred))
print(recall_score(y_valid, knn_pred))
print(f1_score(y_valid, knn_pred))

# Logistic Regression

In [ ]:
from sklearn.linear_model import LogisticRegression
lr = LogisticRegression()
lr.fit(x_train, y_train)
lr_pred= lr.predict(x_valid)

In [ ]:
print(classification_report(y_valid,lr_pred))

In [ ]:
accuracylr = accuracy_score(y_valid, lr_pred)
print("Accuracy = ",accuracylr)

In [ ]:
print(precision_score(y_valid, lr_pred))
print(recall_score(y_valid, lr_pred))
print(f1_score(y_valid, lr_pred))

# Naive Bayes

In [ ]:
from sklearn.naive_bayes import GaussianNB
nb = GaussianNB()
nb.fit(x_train,y_train)
nb_pred= nb.predict(x_valid)

In [ ]:
print(classification_report(y_valid,nb_pred))

In [ ]:
accuracy4 = accuracy_score(y_valid, nb_pred)
print("Accuracy = ",accuracy4)

In [ ]:
print(precision_score(y_valid, nb_pred))
print(recall_score(y_valid, nb_pred))
print(f1_score(y_valid, nb_pred))

# Decision Tree

In [ ]:
from sklearn.tree import DecisionTreeClassifier

In [ ]:
dt = DecisionTreeClassifier(min_samples_leaf=5,max_depth=3, random_state=42)
dt.fit(x_train,y_train)

In [ ]:
dt_pred= dt.predict(x_valid)

In [ ]:
from sklearn.tree import plot_tree
import matplotlib.pyplot as plt

plt.figure(figsize=(10, 6))
plot_tree(dt, filled=True, feature_names=x.columns, class_names=['Class 0', 'Class 1'])
plt.show()


In [ ]:
print(classification_report(y_valid,dt_pred))

In [ ]:
accuracydt = accuracy_score(y_valid, dt_pred)
print("Accuracy = ",accuracydt)

In [ ]:
print(precision_score(y_valid, dt_pred))
print(recall_score(y_valid, dt_pred))
print(f1_score(y_valid, dt_pred))

# Random Forest

In [ ]:
from sklearn.ensemble import RandomForestClassifier

rf_classifier = RandomForestClassifier(n_estimators=100, random_state=42)
rf_classifier.fit(x_train, y_train)


In [ ]:
rf_pred= rf_classifier.predict(x_valid)


In [ ]:
print(classification_report(y_valid,rf_pred))

In [ ]:
accuracyrf = accuracy_score(y_valid, rf_pred)
print("Accuracy = ",accuracyrf)

In [ ]:
from sklearn.model_selection import GridSearchCV

In [ ]:
param_grid = {
    'n_estimators': [100],
    'max_depth':  [3],
    'min_samples_split': [5],
    'min_samples_leaf': [5],
}

grid_search = GridSearchCV(rf_classifier, param_grid, cv=5, scoring='accuracy')

grid_search.fit(x_train, y_train)


In [ ]:
y_predrfg = grid_search.predict(x_valid)

In [ ]:
accuracyrfg = accuracy_score(y_valid, y_predrfg)
print(f'Test Accuracy: {accuracyrfg:.4f}')

In [ ]:
print(classification_report(y_valid,y_predrfg))

In [ ]:
print(precision_score(y_valid,y_predrfg))
print(recall_score(y_valid, y_predrfg))
print(f1_score(y_valid, y_predrfg))